In [ ]:
import datetime, time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from bs4 import BeautifulSoup
import requests

In [ ]:
csv = pd.read_csv('./data/3-15_22.csv')

In [ ]:
csv.columns

In [ ]:
csv['percentage'] = csv['usd_pledged'] / csv['goal']
csv['duration'] = csv['deadline'] - csv['launched_at']

def boolean_to_number(x):
    if x is True:
        return 1
    elif x is False:
        return 0

csv['spotlight_num'] = csv['spotlight'].apply(boolean_to_number)

In [ ]:
target_features = ['usd_pledged', 'backers_count', 'percentage']
input_features = ['goal', 'deadline', 'launched_at', 'duration', 'spotlight_num'] + list(csv.columns[33:-3])

In [ ]:
len(input_features)

In [ ]:
from sklearn import linear_model
from sklearn import preprocessing

x = csv[input_features] # 44930 (5 + 44925)
y = csv[target_features] # 3

print ("... Normalize features")
min_max_scalar = preprocessing.MinMaxScaler()
x_scaled = min_max_scalar.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled)
y_scaled = min_max_scalar.fit_transform(y)
y_scaled = pd.DataFrame(y_scaled)

print ("... Training")
reg = linear_model.Ridge(alpha=.5)
reg.fit(x_scaled, y_scaled)

In [ ]:
coef = pd.DataFrame(reg.coef_, columns=input_features)

In [ ]:
coef

In [ ]:
target_idx = 2
mx = max(reg.coef_[target_idx])
for idx in range(len(reg.coef_[target_idx])):
    if reg.coef_[target_idx][idx] == mx:
        print (idx)

In [ ]:
word_list = []
word_list.extend(open('unigram_list.txt').readlines())
word_list.extend(open('bigram_list.txt').readlines())
word_list.extend(open('trigram_list.txt').readlines())

In [ ]:
def list_view(top_idxs, lst):
    top_n_lst = []
    for idx in top_idxs:
        if idx.isdigit():
            top_n_lst.append(lst[int(idx)])
        else:
            top_n_lst.append(idx)
    return top_n_lst

In [ ]:
print (coef.transpose().nlargest(50, 0)[0].index)
print (coef.transpose().nlargest(50, 0)[0].values)

print (coef.transpose().nsmallest(50, 0)[0].index)
print (coef.transpose().nsmallest(50, 0)[0].values)

In [ ]:
import pickle
# for target_idx in range(3):
target_idx = 
top_n = 50
top_n_idxs = coef.transpose().nsmallest(top_n, target_idx)[target_idx].index
# top_n_idxs = coef.transpose().nlargest(top_n, target_idx)[target_idx].index
print ("target variable: ", target_features[target_idx])
with open('smallest_50_percentage.pickle', 'wb') as handle:
    pickle.dump(list_view(top_n_idxs, word_list), handle, protocol=pickle.HIGHEST_PROTOCOL)
print (list_view(top_n_idxs, word_list))

### Final

In [ ]:
import glob
lst = []
for file in glob.glob('./data_feature/*.csv'):
    df = pd.read_csv(file, index_col=None, header=0)
    lst.append(df)

In [ ]:
df = pd.concat(lst)

In [ ]:
df['percentage'] = df['usd_pledged'] / df['goal']
df['duration'] = df['deadline'] - df['launched_at']

def boolean_to_number(x):
    if x is True:
        return 1
    elif x is False:
        return 0

df['spotlight_num'] = df['spotlight'].apply(boolean_to_number)

In [ ]:
target_features = ['usd_pledged', 'backers_count', 'percentage']
input_features = ['goal', 'deadline', 'launched_at', 'duration', 'spotlight_num'] + list(df.columns[33:-3])

In [ ]:
from sklearn import linear_model

x = df[input_features]
y = df[target_features]

ratio = 0.75
dev_split = int(len(x) * ratio)
train_x = x[:dev_split]
train_y = y[:dev_split]
test_x = x[dev_split:]
test_y = y[dev_split:]

In [ ]:
print ("... Normalize features")
def normalize_features(x):
    from sklearn import preprocessing
    min_max_scalar = preprocessing.MinMaxScaler()    
    return pd.DataFrame(min_max_scalar.fit_transform(x))

train_x_scaled = normalize_features(train_x)
train_y_scaled = normalize_features(train_y)
test_x_scaled = normalize_features(test_x)
test_y_scaled = normalize_features(test_y)

print ("... Training")
reg = linear_model.Ridge(alpha=.5)
reg.fit(train_x_scaled, train_y_scaled)

In [ ]:
reg.score(test_x_scaled, test_y_scaled)

In [ ]:
coef = pd.DataFrame(reg.coef_, columns=input_features)

In [ ]:
print (coef.transpose().nlargest(50, 0)[0].index)
print (coef.transpose().nlargest(50, 0)[0].values)

print (coef.transpose().nsmallest(50, 0)[0].index)
print (coef.transpose().nsmallest(50, 0)[0].values)

In [ ]:
word_list = []
word_list.extend(open('./data_feature/unigram_list.txt').readlines())
word_list.extend(open('./data_feature/bigram_list.txt').readlines())
word_list.extend(open('./data_feature/trigram_list.txt').readlines())

def list_view(top_idxs, lst):
    top_n_lst = []
    for idx in top_idxs:
        if idx.isdigit():
            top_n_lst.append(lst[int(idx)])
        else:
            top_n_lst.append(idx)
    return top_n_lst

In [ ]:
import pickle
# for target_idx in range(3):
target_idx = 0
top_n = 50
# top_n_idxs = coef.transpose().nsmallest(top_n, target_idx)[target_idx].index
top_n_idxs = coef.transpose().nlargest(top_n, target_idx)[target_idx].index
print ("target variable: ", target_features[target_idx])
# with open('smallest_50_percentage.pickle', 'wb') as handle:
#     pickle.dump(list_view(top_n_idxs, word_list), handle, protocol=pickle.HIGHEST_PROTOCOL)
print (list_view(top_n_idxs, word_list))

In [ ]:
dt2 = pd.read_csv('./data_feature/feature_Kickstarter001_2.csv')